In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
X_1_train_path = r"K:\PROJECTS\Mini Projects\S5_BIT_CONNECT\data\seperated\images\violence"
X_0_train_path = r"K:\PROJECTS\Mini Projects\S5_BIT_CONNECT\data\seperated\images\non_violence"
test_path = r"K:\PROJECTS\Mini Projects\S5_BIT_CONNECT\data\raw\Test_image"

In [15]:
import os
import cv2
import numpy as np
def data_injuction(data_path, y):
    files = os.listdir(data_path)
    data = []
    for i, fname in enumerate(files):
        try: 
            path = os.path.join(data_path, fname)
            image = cv2.imread(path)
            roi = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            roi = cv2.resize(roi, (224, 224))
            roi = preprocess_input(roi)
            roi = np.expand_dims(roi, axis=0)
            features = base_model.predict(roi)
            features = features.flatten()
            data.append([features, y])
        except:
            print("An error occured!")
    return data

In [16]:
X_0 = data_injuction(X_0_train_path, 0)
X_1 = data_injuction(X_1_train_path, 1)

1/1 [==============================] - 1s 521ms/step
An error occured!
1/1 [==============================] - 1s 504ms/step
An error occured!
1/1 [==============================] - 1s 512ms/step
An error occured!
1/1 [==============================] - 1s 544ms/step
An error occured!
1/1 [==============================] - 0s 498ms/step
An error occured!
1/1 [==============================] - 0s 490ms/step
An error occured!
1/1 [==============================] - 0s 499ms/step


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split([i[0] for i in X_0]+[i[0] for i in X_1], [i[1] for i in X_0]+[i[1] for i in X_1], test_size=0.33, random_state=42)

In [18]:
from sklearn import svm
model = svm.SVC(probability=True)
model.fit(X_train, y_train)

SVC(probability=True)

In [19]:
y_prid = model.predict(X_test)

In [20]:
from sklearn.metrics import  classification_report
print(classification_report(y_test,y_prid))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       371
           1       0.00      0.00      0.00        34

    accuracy                           0.92       405
   macro avg       0.46      0.50      0.48       405
weighted avg       0.84      0.92      0.88       405



K:\SOFTWARES\APPLICATIONS\ANACONDA3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
K:\SOFTWARES\APPLICATIONS\ANACONDA3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
K:\SOFTWARES\APPLICATIONS\ANACONDA3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
import joblib
model=joblib.load("post_detection.joblib")

In [21]:
single_test_path = r"K:\PROJECTS\Mini Projects\S5_BIT_CONNECT\data\raw\Test_image\violence\violence31.jpg"
img = cv2.imread(single_test_path)
roi = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
roi = cv2.resize(roi, (224, 224))
roi = preprocess_input(roi)
roi = np.expand_dims(roi, axis=0)
features = base_model.predict(roi)
features = features.flatten()
output = model.predict([features])
if output==1:print("Non-violance")
else: print("Violance")

1/1 [==============================] - 0s 440ms/step
Non-violance


In [ ]:

# Save the One-Class SVM model
model_filename = 'post_detection.joblib'
joblib.dump(model, model_filename)